# BindsNet Encoders

## 1. Table of Contents
1. Table of Contents
2. Overview
2. Import Statements
3. Encoders
    1. Single Spike Encoders
    2. Repeat Encoder
    3. Poisson Encoders
    4. Bernoulli Encoders
    5. Rank Order Encoder

## 2. Overview
All of the encoders below transform an N dimensional input tensor into an N + 1 dimensional tensor, where the added dimension is time. The data fed into the encoders must all be floating point values. This is because some of the encoders perform divisions using the data that could approximate to fractional values.

## 2. Import Statements

In [13]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

from time import time as t

from bindsnet.encoding import *

## 3a. Single (Spike) Encoder
### Summary
The single (spike) encoder transforms the data into a tensor where the value at time 0 in the tensor for a given element in the input has a spike if the value is greater than the quantile cutoff. The closer to 1 the sparsity is, the more spikes will be permitted. A value of 0 means no spikes will be permitted.
### Quantile Calculation
For Numpy, the quantile is calculated as:
```
np.quantile(np.array([4, 3, 2, 1]),0) 
1
np.quantile(np.array([4, 3, 2, 1]),0.25) 
1.75
np.quantile(np.array([4, 3, 2, 1]),0.50) 
2.5
np.quantile(np.array([4, 3, 2, 1]),0.75) 
3.25
np.quantile(np.array([4, 3, 2, 1]),1) 
4
```

In [14]:
# time is the total sample duration in ms
# set to 1 for this example because steps after the first will not generate spikes
time = 1

# dt is the step size within the sample duration in ms
dt = 1.0

# input data
train_image = torch.FloatTensor([[80,70,60],[50,40,30],[20,10,0]])

# print train data
print("Input Datum:")
print(train_image)

# sparsity of spikes (1 for all spikes, 0 for no spikes)

# spikes over the 0.75 quantile will be permitted
sparsity = 0.25
encoder = SingleEncoder(time=time, dt=dt,sparsity=sparsity)
encoded_image = encoder(train_image)
print("Encoded Datum w/ Sparsity of 0.25:")
print(encoded_image)

# spikes over the 0.5 quantile will be permitted
sparsity = 0.5
encoder = SingleEncoder(time=time, dt=dt,sparsity=sparsity)
encoded_image = encoder(train_image)
print("Encoded Datum w/ Sparsity of 0.5:")
print(encoded_image)

# spikes over the 0.25 quantile will be permitted
sparsity = 0.75
encoder = SingleEncoder(time=time, dt=dt,sparsity=sparsity)
encoded_image = encoder(train_image)
print("Encoded Datum w/ Sparsity of 0.75:")
print(encoded_image)

# all spikes will be permitted
sparsity = 1
encoder = SingleEncoder(time=time, dt=dt,sparsity=sparsity)
encoded_image = encoder(train_image)
print("Encoded Datum w/ Sparsity of 1:")
print(encoded_image)

Input Datum:
tensor([[80., 70., 60.],
        [50., 40., 30.],
        [20., 10.,  0.]])
Encoded Datum w/ Sparsity of 0.25:
tensor([[[1, 1, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=torch.uint8)
Encoded Datum w/ Sparsity of 0.5:
tensor([[[1, 1, 1],
         [1, 0, 0],
         [0, 0, 0]]], dtype=torch.uint8)
Encoded Datum w/ Sparsity of 0.75:
tensor([[[1, 1, 1],
         [1, 1, 1],
         [0, 0, 0]]], dtype=torch.uint8)
Encoded Datum w/ Sparsity of 1:
tensor([[[1, 1, 1],
         [1, 1, 1],
         [1, 1, 0]]], dtype=torch.uint8)


## 3b. Repeat Encoder
### Summary
Repeats the same datum vector for `time/dt` timesteps


In [15]:
# time is the total sample duration in ms
time = 3

# dt is the step size within the sample duration in ms
dt = 1.0

# input data
train_image = torch.FloatTensor([[80,70,60],[50,40,30],[20,10,0]])

# print train data
print("Input Datum:")
print(train_image)

# create encoder and encode input
encoder = RepeatEncoder(time=time, dt=dt)
train_image = torch.FloatTensor([[80,70,60],[50,40,30],[20,10,0]])
encoded_image = encoder(train_image)
print("Encoded Datum:")
print(encoded_image)

Input Datum:
tensor([[80., 70., 60.],
        [50., 40., 30.],
        [20., 10.,  0.]])
Encoded Datum:
tensor([[[80., 70., 60.],
         [50., 40., 30.],
         [20., 10.,  0.]],

        [[80., 70., 60.],
         [50., 40., 30.],
         [20., 10.,  0.]],

        [[80., 70., 60.],
         [50., 40., 30.],
         [20., 10.,  0.]]])


## 3c. Poisson Encoder

### Summary
Generates Poisson-distributed spike trains based on input intensity. Inputs must be non-negative, and give the firing rate in Hz. Inter-spike intervals (ISIs) for non-negative data incremented by one to avoid zero intervals while maintaining ISI distributions.

### Limitations
* Inputs must be non-negative
* Inputs must be floating point numbers (otherwise the rate will zero out)

### Rate Calculation

1. Compute firing rates in seconds as a function of data intensity
    * rate = (1 / input_intensity) * (1000 / dt)
    * higher intensity -> lower inter-spike interaval
    * dt = simulation time step
2. Create Poisson distribution and sample inter-spike intervals
    * `torch.distributions.Poisson()` used to generate samples
    * `time/dt` samples are taken for each input element based on the probability density function (pdf) of a Poisson distribution
        * Vales near the rate (lambda) more likely to occur
        * The higher the rate, the more varied the data will be
    * if any samples produced 0, set them to 1
3. Incrementally add all of the samples generated for each input element
    * this produces indexes of where to set the spike outputs for each element in the spikes array
    * if any samples are greater than `time/dt`, set them to 0 because they are outside of the sample duration
4. Create tensor of spikes
    * for each of the indexes produced, set the corresponding indexes in the spike tensor to 1
    * given an input tensor with dimsnions X x Y, the output tensor has dimensions (time/dt + 1) x X x Y
    * remove the first row of spikes because they are all set to 1


### Poisson Distribution

![Poisson Probability Function](https://wikimedia.org/api/rest_v1/media/math/render/svg/c22cb4461e100a6db5f815de1f44b1747f160048)

![Poisson Probability Graph Function](https://upload.wikimedia.org/wikipedia/commons/thumb/1/16/Poisson_pmf.svg/325px-Poisson_pmf.svg.png)

where
* k is the number of occurrences
* λ is the rate (mean number of occurrences in the interval)

(Source [wikipedia.org](https://en.wikipedia.org/wiki/Poisson_distribution))

In [16]:
# time is the total sample duration in ms
time = 10

# dt is the step size within the sample duration in ms
dt = 1.0

# input data
train_image = torch.FloatTensor([[80,70,60],[50,40,30],[20,10,0]])

# print train data
print("Input Datum:")
print(train_image)

# create encoder and encode input
encoder = PoissonEncoder(time=time, dt=dt)
encoded_image = encoder(train_image)
print("Encoded Datum:")
print(encoded_image)

Input Datum:
tensor([[80., 70., 60.],
        [50., 40., 30.],
        [20., 10.,  0.]])
Encoded Datum:
tensor([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[1, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=torch.uint8)


## 3d. Bernoulli Encoder
### Summary
Generates Bernoulli-distributed spike trains based on input intensity. Inputs must be non-negative. Spikes correspond to successful Bernoulli trials, with success probability equal to (normalized in \[0, 1]) input value.

### Limitations
* Inputs must be non-negative
* Inputs must be floating point numbers (otherwise the rate will zero out)

### Bernoulli Trial Calculation
1. Inputs are normalized based on the max intensity
    * The highest intensity will have a probability of 1 and always spike
    * An intensity of 0 will have a probability of 0 and never spike
2. The spike tensor is generated using `torch.bernoulli()` to generate the random samples
    * Highest intensity will always produce 1, lower intensities will generate 1's based on their scale against the highest intensity

In [17]:
# time is the total sample duration in ms
time = 10

# dt is the step size within the sample duration in ms
dt = 1.0

# input data
train_image = torch.FloatTensor([[80,70,60],[50,40,30],[20,10,0]])

# print train data
print(train_image)

# create encoder and encode input
encoder = BernoulliEncoder(time=time, dt=dt)
encoded_image = encoder(train_image)
print(encoded_image)

tensor([[80., 70., 60.],
        [50., 40., 30.],
        [20., 10.,  0.]])
tensor([[[1, 1, 1],
         [1, 0, 0],
         [0, 0, 0]],

        [[1, 1, 0],
         [1, 1, 1],
         [0, 0, 0]],

        [[1, 1, 1],
         [0, 0, 0],
         [0, 0, 0]],

        [[1, 1, 1],
         [0, 1, 1],
         [1, 0, 0]],

        [[1, 1, 1],
         [0, 1, 0],
         [1, 1, 0]],

        [[1, 1, 1],
         [1, 1, 1],
         [0, 1, 0]],

        [[1, 1, 1],
         [0, 0, 1],
         [1, 1, 0]],

        [[1, 1, 1],
         [0, 0, 0],
         [1, 0, 0]],

        [[1, 1, 1],
         [1, 1, 1],
         [0, 0, 0]],

        [[1, 1, 0],
         [1, 1, 0],
         [0, 0, 0]]], dtype=torch.uint8)


## 3e. Rank Order Encoder
* data with the same value will spike at the same time
* first: data scaled based on the max value
* times array created with highest value having value of 1 and lowest value closer to inf
* time array multiplied by num timesteps / max value in time array
* values are rounded up to next whole number
* spike array created n elements for each timestep
* iterate through each element i
* if the rank for i is within the time number of timesteps
* add a spike for this element wherever the spike is supposed to occur
* the close the data is to 0, the higher (later) the rank will be

In [18]:
# time is the total sample duration in ms
time = 10

# dt is the step size within the sample duration in ms
dt = 1.0

# input data
train_image = torch.FloatTensor([[80,70,60],[50,40,30],[20,10,0]])

# print train data
print(train_image)

# create encoder and encode input
encoder = RankOrderEncoder(time=time, dt=dt)
encoded_image = encoder(train_image)
print(encoded_image)

tensor([[80., 70., 60.],
        [50., 40., 30.],
        [20., 10.,  0.]])
tensor([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[1, 1, 1],
         [1, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 1, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 1],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [1, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=torch.uint8)
